In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from wordcloud import WordCloud
import re

import matplotlib.pyplot as plt 
import seaborn as sns
from wordcloud import WordCloud
import pyLDAvis
sns.set()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import gensim


# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
# from nltk.stem.porter import PorterStemmer
# from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import stopwords

E:\Anaconda\envs\myenvnew\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
E:\Anaconda\envs\myenvnew\lib\site-packages\gensim\matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [3]:
df = pd.read_csv(r'C:/Users/pande/Job-Description-Skills-Extractor/data/collected_data/df_description_processed.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5379 entries, 0 to 5378
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Description         5379 non-null   object
 1   lower_description   5379 non-null   object
 2   word_tokenized      5379 non-null   object
 3   sentence_tokenized  5379 non-null   object
 4   word_count          5379 non-null   int64 
 5   sentence_count      5379 non-null   int64 
 6   clean_words         5379 non-null   object
 7   clean_stemmed       5379 non-null   object
 8   clean_lemmed        5379 non-null   object
dtypes: int64(2), object(7)
memory usage: 378.3+ KB


In [4]:
# create the transform
vectorizer = CountVectorizer(stop_words = 'english', max_df = 0.75,ngram_range=(2, 4))
# tokenize and build vocab
bag_of_words = vectorizer.fit_transform(df.clean_lemmed)
# summarize
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vectorizer.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
words_freq[:10]

[('machine learning', 5219),
 ('year experience', 4067),
 ('data science', 3393),
 ('big data', 3053),
 ('computer science', 2885),
 ('data scientist', 2560),
 ('data analysis', 2069),
 ('communication skill', 1972),
 ('bachelor degree', 1966),
 ('experience working', 1897)]

In [5]:
# 5379 documents represented as a 14,409 dimensional vector (14,409 words)
bag_of_words

<5379x1876070 sparse matrix of type '<class 'numpy.int64'>'
	with 3933900 stored elements in Compressed Sparse Row format>

In [6]:
LDA = LatentDirichletAllocation(n_components=5, random_state=42)
LDA.fit(bag_of_words)

LatentDirichletAllocation(n_components=5, random_state=42)

In [7]:
first_topic = LDA.components_[0]

In [8]:
top_topic_words = first_topic.argsort()[-10:]
top_topic_words

array([ 396398,  656593,  167167,  675700,  434532,  189813,  432340,
        328216, 1002591, 1869757], dtype=int64)

In [9]:
for i in top_topic_words:
    print(vectorizer.get_feature_names()[i])

E:\Anaconda\envs\myenvnew\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


data analysis
experience data
bachelor degree
experience working
data scientist
big data
data science
computer science
machine learning
year experience


In [10]:
for i,topic in enumerate(LDA.components_):
    print(f'Top 10 words for topic #{i}:')
    print([vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for topic #0:
['data analysis', 'experience data', 'bachelor degree', 'experience working', 'data scientist', 'big data', 'data science', 'computer science', 'machine learning', 'year experience']


Top 10 words for topic #1:
['experience working', 'communication skill', 'data analysis', 'data set', 'computer science', 'data scientist', 'big data', 'year experience', 'data science', 'machine learning']


Top 10 words for topic #2:
['experience working', 'communication skill', 'data scientist', 'bachelor degree', 'data analysis', 'big data', 'computer science', 'data science', 'year experience', 'machine learning']


Top 10 words for topic #3:
['experience working', 'data analysis', 'bachelor degree', 'communication skill', 'data scientist', 'computer science', 'data science', 'big data', 'year experience', 'machine learning']


Top 10 words for topic #4:
['communication skill', 'bachelor degree', 'data analysis', 'data analytics', 'data scientist', 'year experience', 'comp

In [11]:
topic_values = LDA.transform(bag_of_words)
topic_values.shape

(5379, 5)

In [12]:
df['Topic'] = topic_values.argmax(axis=1)

In [13]:
df.head()

,Description,lower_description,word_tokenized,sentence_tokenized,word_count,sentence_count,clean_words,clean_stemmed,clean_lemmed,Topic
0,"POSITION SUMMARY, The Business Analyst role is...","position summary, the business analyst role is...","['position', 'summary', 'the', 'business', 'an...","['POSITION SUMMARY, The Business Analyst role ...",424,25,"['position', 'summary', 'business', 'analyst',...","['posit', 'summari', 'busi', 'analyst', 'role'...","['position', 'summary', 'business', 'analyst',...",3
1,"What do we need?, You to have an amazing perso...","what do we need?, you to have an amazing perso...","['what', 'do', 'we', 'need', 'you', 'to', 'hav...","['What do we need?, You to have an amazing per...",286,10,"['need', 'amazing', 'personality', 'communicat...","['need', 'amaz', 'person', 'commun', 'style', ...","['need', 'amazing', 'personality', 'communicat...",4
2,"Validate, analyze, and conduct statistical ana...","validate, analyze, and conduct statistical ana...","['validate', 'analyze', 'and', 'conduct', 'sta...","['Validate, analyze, and conduct statistical a...",314,24,"['validate', 'analyze', 'conduct', 'statistica...","['valid', 'analyz', 'conduct', 'statist', 'ana...","['validate', 'analyze', 'conduct', 'statistica...",0
3,"Full time, Washington, DC metro area, Starting...","full time, washington, dc metro area, starting...","['full', 'time', 'washington', 'dc', 'metro', ...","['Full time, Washington, DC metro area, Starti...",297,13,"['full', 'time', 'washington', 'dc', 'metro', ...","['full', 'time', 'washington', 'dc', 'metro', ...","['full', 'time', 'washington', 'dc', 'metro', ...",2
4,Assist in consultations with business partners...,assist in consultations with business partners...,"['assist', 'in', 'consultations', 'with', 'bus...",['Assist in consultations with business partne...,316,7,"['assist', 'consultations', 'business', 'partn...","['assist', 'consult', 'busi', 'partner', 'inte...","['assist', 'consultation', 'business', 'partne...",4
